# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [64]:
import warnings
warnings.filterwarnings('ignore')

In [65]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [66]:
#!pip install pandas

In [68]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [69]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [70]:
pip install langchain_community

  Using cached langchain_community-0.3.21-py3-none-any.whl.metadata (2.4 kB)
  Using cached aiohttp-3.11.16-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.4.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached propcache-0.3.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached yarl-1.19.0-cp313-cp313-win_amd64.whl.metadata (74 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 k

In [71]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [72]:
from dotenv import load_dotenv, find_dotenv
import os
import openai

load_dotenv(find_dotenv())  # Load environment variables from .env file

openai.api_key = os.getenv('OPENAI_API_KEY')


In [73]:
#Replace None by your own value and justify

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo", openai_api_key=openai.api_key)

# temperature=0.7, which is a balanced setting that provides a mix of creativity and coherence in responses.

In [74]:
prompt = ChatPromptTemplate.from_template(
    "Describe the features and benefits of the following product: {product_name}. What are its main selling points?"
)

In [75]:

chain = LLMChain(llm=llm, prompt=prompt)

In [76]:
product = "smartphone" #Select a product type to be describe
chain.run(product)

"A smartphone is a handheld device that combines the functionalities of a mobile phone with those of a computer. Some of the key features and benefits of a smartphone include:\n\n1. Communication: Smartphones allow users to make phone calls, send text messages, and access various messaging apps to stay connected with friends, family, and colleagues.\n\n2. Internet access: Smartphones provide high-speed internet access through Wi-Fi or mobile data, allowing users to browse the web, check emails, and access social media platforms.\n\n3. Apps: Smartphones come with a wide range of apps that can be downloaded from app stores, offering users access to games, productivity tools, entertainment, and much more.\n\n4. Camera: Most smartphones come with high-quality cameras that allow users to capture photos and videos on the go, edit them, and share them instantly with others.\n\n5. Multimedia: Smartphones are equipped with features that allow users to listen to music, watch videos, and stream c

## SimpleSequentialChain

In [77]:
from langchain.chains import SimpleSequentialChain

In [78]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe the features and benefits of the following product: {product_name}. What are its main selling points?")



# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [80]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description, write a short and catchy advertisement: {product_description}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [81]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [82]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
A smartphone is a handheld device that combines the functionality of a mobile phone with that of a computer. It typically features a touchscreen interface, internet connectivity, and a variety of applications that can be downloaded and installed to enhance its functionality.

Some key features of a smartphone include:

1. Communication: Smartphones allow users to make calls, send text messages, and access email, making it easy to stay connected with friends, family, and colleagues.

2. Internet connectivity: Smartphones can connect to Wi-Fi networks and cellular data networks, allowing users to browse the web, check social media, and access online services from anywhere.

3. Apps: Smartphones support a wide range of applications, or "apps," that can be downloaded and installed to perform specific tasks, such as games, productivity tools, navigation, and more.

4. Camera: Most smartphones come equipped with high-quality cameras that allow 

'"Stay connected, entertained, and productive on the go with the ultimate all-in-one device - the smartphone! With communication, internet connectivity, apps, a high-quality camera, multimedia capabilities, and personalization options, the smartphone is the perfect tool to enhance your daily life. Get yours today and experience the convenience and versatility of modern technology at your fingertips!"'

**Repeat the above twice for different products**

## SequentialChain

In [83]:
from langchain.chains import SequentialChain

In [84]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following customer review to English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translated_review" #Give a name to your output
                    )


In [87]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following customer review in a concise paragraph:\n\n{translated_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary" #give a name to this output
                    )


In [88]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
   "Identify the language of the following text:\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [89]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
       "Given the review in {language}, and its translation: {translated_review}, provide a thoughtful follow-up response."

)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_response"
                     )


In [90]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["translated_review", "summary", "follow_up_response"],
    verbose=True
)

In [91]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't last, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The customer finds the taste of the product mediocre and notes that the foam does not last, which they find strange. They suspect that the product may be an old batch or counterfeit, as they have bought the same product in stores and found the taste to be much better.',
 'follow_up_response': "I'm sorry to hear that you were disappointed with the taste of the product. It's possible that you may have received a defective batch or a counterfeit item, as the taste should not be significantly different from the ones you normally purchase in stores. I recommend reaching out to the manufacturer or retailer to address your concern

**Repeat the above twice for different products or reviews**

In [96]:
review_2 = df.Review[2]  
overall_chain(review_2)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'translated_review': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mat

In [98]:
review_2 = df.Review[4]  
overall_chain(review_2)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'translated_review': "I loved this product. However, it seems to only last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). After 4 months, my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand it, they will only replace it once. Anyway, if you have one, good luck.",
 'summary': 'The customer loved the product but was disappointed that it only lasted a few months before the frother

## Router Chain

In [99]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [100]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [101]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [102]:
llm = ChatOpenAI(temperature=0)

In [103]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [104]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [105]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [106]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [107]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [108]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [109]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [110]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [111]:
# Running the chain with a computer science question
chain.run("Explain how an algorithm for binary search works.")



> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain how an algorithm for binary search works.'}
> Finished chain.


'Binary search is a classic algorithm used to efficiently search for a target value within a sorted array. The algorithm works by repeatedly dividing the search interval in half until the target value is found or the interval is empty.\n\nHere is how the binary search algorithm works:\n\n1. Start with defining the search interval as the entire array.\n2. Calculate the middle index of the search interval.\n3. Compare the target value with the value at the middle index.\n4. If the target value is equal to the middle value, the search is successful and the index of the target value is returned.\n5. If the target value is less than the middle value, repeat the search on the left half of the array (excluding the middle value).\n6. If the target value is greater than the middle value, repeat the search on the right half of the array (excluding the middle value).\n7. Repeat steps 2-6 until the target value is found or the search interval is empty.\n\nThe key advantage of binary search is that

In [112]:
# Running the chain with a biology question
chain.run("What is the process of photosynthesis?")




> Entering new MultiPromptChain chain...
biology: {'input': 'What is the process of photosynthesis?'}
> Finished chain.


'Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy stored in glucose molecules. This process occurs in the chloroplasts of plant cells and involves several steps:\n\n1. Light absorption: Chlorophyll, a pigment found in chloroplasts, absorbs light energy from the sun.\n\n2. Water splitting: Water molecules are split into oxygen, protons, and electrons by a process called photolysis. The oxygen is released as a byproduct, while the protons and electrons are used in the next steps of photosynthesis.\n\n3. Electron transport chain: The electrons from water are passed through a series of proteins in the thylakoid membrane of the chloroplast, creating a proton gradient across the membrane.\n\n4. ATP synthesis: The proton gradient drives the synthesis of ATP, a molecule that stores energy for cellular processes.\n\n5. Carbon fixation: Carbon dioxide from the atmosphere is taken up by the plant and con

In [113]:
# Running the chain with a physics question
chain.run("What is the law of conservation of energy?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is the law of conservation of energy?'}
> Finished chain.


'The law of conservation of energy states that energy cannot be created or destroyed, only transferred or transformed from one form to another. In other words, the total energy in a closed system remains constant over time. This principle is a fundamental concept in physics and is used to analyze and understand various physical phenomena.'